In [1]:
import pandas as pd
import numpy as np

### 完成内容  
1. 清洗JURISDICTION_CODE列，Police(0), Transit(1), and Housing(2); or external(3)
2. 删除PARKS_NM，99.25为空值
3. 重新标记性别（VIC&SUSP） male-1 female-2 other-3 Nan-0
4. 重新标记种族（VIC&SUSP） Nan&Unknown-0 BLACK-1  BLACK HISPANIC-2  WHITE-3  WHITE HISPANIC- 4 ASIAN/PACIFIC ISLANDER-5 AMERICAN INDIAN/ALASKAN NATIVE-6
5. 检查Latitude，Longitude列 与 Lat_Lon列是否一一对应
6. 重新标记犯罪场所相对位置LOC_OF_OCCUR_DESC 'INSIDE':1, 'FRONT OF':2, 'OPPOSITE OF':3 , 'REAR OF':4  
    PREM_TYP_DESC为犯罪场所, 共74种类，未处理（string类型）
7. 建立犯罪label与犯罪类型的对应关系（细粒度与粗粒度），并分组统计（粗粒度：'KY_CD'=>"OFNS_DESC", 细粒度'PD_CD'=>"PD_DESC")
8. 重新标记犯罪是否成功(CRM_ATPT_CPTD_CD) 'COMPLETED':1, 'ATTEMPTED':2
9. 重新标记犯罪严重程度（LAW_CAT_CD） 'FELONY':1, 'VIOLATION':2, 'MISDEMEANOR':3

In [2]:
df = pd.read_csv('preprocess_311.csv')

In [3]:
del df['Unnamed: 0']

In [4]:
df.CMPLNT_FR_DT = pd.to_datetime(df.CMPLNT_FR_DT)
df.CMPLNT_FR_TM = pd.to_datetime(df.CMPLNT_FR_TM)

In [5]:
df.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,...,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,Latitude,Longitude,Lat_Lon,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,708871766,2021-12-23,1900-01-01 13:30:00,126,MISCELLANEOUS PENAL LAW,640.0,AGGRAVATED HARASSMENT 1,COMPLETED,FELONY,BRONX,...,241853.0,UNKNOWN,UNKNOWN,M,40.830443,-73.871349,"(40.83044253800006, -73.871349147)",18-24,BLACK,M
1,498849139,2019-12-31,1900-01-01 01:00:00,578,HARRASSMENT 2,638.0,"HARASSMENT,SUBD 3,4,5",COMPLETED,VIOLATION,QUEENS,...,156860.0,UNKNOWN,BLACK,M,40.596994,-73.767859,"(40.59699444300003, -73.76785904499997)",45-64,BLACK,F
2,123171799,2021-12-31,1900-01-01 21:15:00,106,FELONY ASSAULT,109.0,"ASSAULT 2,1,UNCLASSIFIED",COMPLETED,FELONY,BRONX,...,237277.0,UNKNOWN,WHITE HISPANIC,M,40.817877,-73.865994,"(40.81787660100008, -73.86599391299995)",65+,BLACK,M
3,725872999,2021-12-21,1900-01-01 23:00:00,114,ARSON,264.0,"ARSON, MOTOR VEHICLE 1 2 3 & 4",COMPLETED,FELONY,BRONX,...,252550.0,NaN,NaN,NaN,40.859744,-73.826259,"(40.859743612000045, -73.82625891199996)",UNKNOWN,UNKNOWN,E
4,432371969,2021-12-31,1900-01-01 18:20:00,113,FORGERY,725.0,"FORGERY,M.V. REGISTRATION",COMPLETED,FELONY,QUEENS,...,211420.0,18-24,WHITE HISPANIC,M,40.746775,-73.780867,"(40.74677541300008, -73.78086726799995)",UNKNOWN,UNKNOWN,E


In [6]:
df.describe()

,CMPLNT_NUM,KY_CD,PD_CD,JURISDICTION_CODE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
count,1.307203e+06,1.307203e+06,1.307203e+06,1.307203e+06,1.307203e+06,1.307203e+06,1.307203e+06,1.307203e+06
mean,5.499413e+08,3.060607e+02,3.972663e+02,6.759539e-01,1.005456e+06,2.078022e+05,4.073700e+01,-7.392345e+01
std,2.598241e+08,1.588825e+02,2.144835e+02,6.724927e+00,2.124784e+04,3.015338e+04,8.276781e-02,7.662898e-02
min,1.000004e+08,1.020000e+02,1.000000e+02,0.000000e+00,9.134110e+05,1.211310e+05,4.049891e+01,-7.425474e+01
25%,3.246092e+08,1.170000e+02,2.540000e+02,0.000000e+00,9.922970e+05,1.852460e+05,4.067507e+01,-7.397099e+01
50%,5.502648e+08,3.410000e+02,3.490000e+02,0.000000e+00,1.004715e+06,2.070540e+05,4.073498e+01,-7.392615e+01
75%,7.750460e+08,3.510000e+02,6.370000e+02,0.000000e+00,1.017179e+06,2.353120e+05,4.081252e+01,-7.388119e+01
max,9.999989e+08,8.810000e+02,9.690000e+02,9.700000e+01,1.067226e+06,2.718200e+05,4.091272e+01,-7.370057e+01


In [7]:
# Police(0), Transit(1), and Housing(2); or external(3)
df = df[df['JURISDICTION_CODE'].isin([0,1,2,3])]

In [8]:
((df.isnull().sum())/df.shape[0]).sort_values(ascending=False).map(lambda x:"{:.2%}".format(x))

PARKS_NM             99.30%
SUSP_SEX             21.88%
SUSP_RACE            21.88%
SUSP_AGE_GROUP       21.88%
LOC_OF_OCCUR_DESC    16.91%
PREM_TYP_DESC         0.22%
CRM_ATPT_CPTD_CD      0.01%
VIC_RACE              0.01%
OFNS_DESC             0.00%
VIC_SEX               0.00%
VIC_AGE_GROUP         0.00%
Y_COORD_CD            0.00%
Lat_Lon               0.00%
Longitude             0.00%
Latitude              0.00%
CMPLNT_NUM            0.00%
X_COORD_CD            0.00%
CMPLNT_FR_DT          0.00%
JURIS_DESC            0.00%
BORO_NM               0.00%
LAW_CAT_CD            0.00%
PD_DESC               0.00%
PD_CD                 0.00%
KY_CD                 0.00%
CMPLNT_FR_TM          0.00%
JURISDICTION_CODE     0.00%
dtype: object

In [9]:
# Nan占比过多
del df['PARKS_NM']

In [10]:
def count_freq(c):
    print(c.agg(['value_counts']).T)

In [11]:
# 统一性别 M-male-1 F-female-2 O-other-3 Nan-0
df.loc[df['SUSP_SEX']=='F','SUSP_SEX'] = 2
df.loc[df['SUSP_SEX']=='M','SUSP_SEX'] = 1
df.loc[df['SUSP_SEX']=='U','SUSP_SEX'] = 3
df['SUSP_SEX'] = df['SUSP_SEX'].fillna(0)
df.SUSP_SEX.unique()

array([1, 0, 3, 2], dtype=int64)

In [12]:
# 统一性别 M-male-1 F-female-2 O-other-3 Nan-0
df.loc[df['VIC_SEX']=='F','VIC_SEX'] = 2
df.loc[df['VIC_SEX']=='M','VIC_SEX'] = 1
df.loc[df['VIC_SEX'].isin(['E','D']),'VIC_SEX'] = 3
df['VIC_SEX'] = df['VIC_SEX'].fillna(0)
df.VIC_SEX.unique()

array([1, 2, 3, 0], dtype=int64)

In [13]:
# 统一种族 Nan&Unknown-0 BLACK-1  BLACK HISPANIC-2  WHITE-3   WHITE HISPANIC- 4
# ASIAN/PACIFIC ISLANDER-5 AMERICAN INDIAN/ALASKAN NATIVE-6
df.loc[df['VIC_RACE']=='UNKNOWN','VIC_RACE'] = 0
df.loc[df['VIC_RACE']=='BLACK','VIC_RACE'] = 1
df.loc[df['VIC_RACE']=='BLACK HISPANIC','VIC_RACE'] = 2
df.loc[df['VIC_RACE']=='WHITE','VIC_RACE'] = 3
df.loc[df['VIC_RACE']=='WHITE HISPANIC','VIC_RACE'] = 4
df.loc[df['VIC_RACE']=='ASIAN / PACIFIC ISLANDER','VIC_RACE'] = 5
df.loc[df['VIC_RACE']=='AMERICAN INDIAN/ALASKAN NATIVE','VIC_RACE'] = 6
df['VIC_RACE'] = df['VIC_RACE'].fillna(0)
count_freq(df['VIC_RACE'])

                   0       1       4       3       5      2     6
value_counts  363555  337710  232864  201542  102670  54910  5424


In [14]:
df.loc[df['SUSP_RACE']=='UNKNOWN','SUSP_RACE'] = 0
df.loc[df['SUSP_RACE']=='BLACK','SUSP_RACE'] = 1
df.loc[df['SUSP_RACE']=='BLACK HISPANIC','SUSP_RACE'] = 2
df.loc[df['SUSP_RACE']=='WHITE','SUSP_RACE'] = 3
df.loc[df['SUSP_RACE']=='WHITE HISPANIC','SUSP_RACE'] = 4
df.loc[df['SUSP_RACE']=='ASIAN / PACIFIC ISLANDER','SUSP_RACE'] = 5
df.loc[df['SUSP_RACE']=='AMERICAN INDIAN/ALASKAN NATIVE','SUSP_RACE'] = 6
df['SUSP_RACE'] = df['SUSP_RACE'].fillna(0)
count_freq(df['SUSP_RACE'])

                   0       1       4      3      2      5     6
value_counts  575800  371797  164572  95169  52177  36743  2417


In [15]:
# 统一年龄组 Nan&Unknow&Other-0   <18-1   18-24-2  25-44-3   45-64-4   65+-5
age_group = {'<18':1, '18-24':2, '25-44':3 , '45-64':4, '65+':5}
df['VIC_AGE_GROUP'] = df['VIC_AGE_GROUP'].apply(lambda x: age_group[x] if (x in age_group.keys()) else 0)
df['SUSP_AGE_GROUP'] = df['SUSP_AGE_GROUP'].apply(lambda x: age_group[x] if (x in age_group.keys()) else 0)
count_freq(df['VIC_AGE_GROUP'])
count_freq(df['SUSP_AGE_GROUP'])

                   3       0       4       2      5      1
value_counts  482379  336628  256419  119697  59060  44492
                   0       3       4      2      1      5
value_counts  726725  325483  113541  97036  24699  11191


In [16]:
# Check Lat Lon
'''for i in range(df.shape[0]):
    lat_lon = df.Lat_Lon[i].split(',')
    lat = float(lat_lon[0][1:])
    lon = float(lat_lon[1][1:-1])
    if (abs(lat - float(df.Latitude[i]) > pow(0.1,6))) | (abs(lon - float(df.Longitude[i]))>pow(0.1,6)) :
        print(lat,df.Latitude[i])'''

"for i in range(df.shape[0]):\n    lat_lon = df.Lat_Lon[i].split(',')\n    lat = float(lat_lon[0][1:])\n    lon = float(lat_lon[1][1:-1])\n    if (abs(lat - float(df.Latitude[i]) > pow(0.1,6))) | (abs(lon - float(df.Longitude[i]))>pow(0.1,6)) :\n        print(lat,df.Latitude[i])"

In [17]:
loc_desc = {'INSIDE':1, 'FRONT OF':2, 'OPPOSITE OF':3 , 'REAR OF':4}
df['LOC_OF_OCCUR_DESC'] = df['LOC_OF_OCCUR_DESC'].apply(lambda x: loc_desc[x] if (x in loc_desc.keys()) else 0)

In [18]:
len(df.PREM_TYP_DESC.unique())

75

In [19]:
# 粗粒度分类
OFNS_key_value = df.groupby(by = ['KY_CD',"OFNS_DESC"])['CMPLNT_NUM'].count()

In [20]:
# 细粒度分类
PD_key_value = df.groupby(by = ['PD_CD',"PD_DESC"])['CMPLNT_NUM'].count()

In [21]:
crm_cd = {'COMPLETED':1, 'ATTEMPTED':2}
df['CRM_ATPT_CPTD_CD'] = df['CRM_ATPT_CPTD_CD'].apply(lambda x: crm_cd[x] if (x in crm_cd.keys()) else 0)

In [22]:
law_cd = {'FELONY':1, 'VIOLATION':2, 'MISDEMEANOR':3}
df['LAW_CAT_CD'] = df['LAW_CAT_CD'].apply(lambda x: law_cd[x] if (x in law_cd.keys()) else 0)

In [23]:
count_freq(df['CRM_ATPT_CPTD_CD'])

                    1      2    0
value_counts  1278241  20331  103


In [24]:
count_freq(df['LAW_CAT_CD'])

                   3       1       2
value_counts  671480  414267  212928


In [25]:
df.dtypes

CMPLNT_NUM                    int64
CMPLNT_FR_DT         datetime64[ns]
CMPLNT_FR_TM         datetime64[ns]
KY_CD                         int64
OFNS_DESC                    object
PD_CD                       float64
PD_DESC                      object
CRM_ATPT_CPTD_CD              int64
LAW_CAT_CD                    int64
BORO_NM                      object
LOC_OF_OCCUR_DESC             int64
PREM_TYP_DESC                object
JURIS_DESC                   object
JURISDICTION_CODE           float64
X_COORD_CD                  float64
Y_COORD_CD                  float64
SUSP_AGE_GROUP                int64
SUSP_RACE                     int64
SUSP_SEX                      int64
Latitude                    float64
Longitude                   float64
Lat_Lon                      object
VIC_AGE_GROUP                 int64
VIC_RACE                      int64
VIC_SEX                       int64
dtype: object

In [31]:
df['OFNS_DESC'].unique()

array(['MISCELLANEOUS PENAL LAW', 'HARRASSMENT 2', 'FELONY ASSAULT',
       'ARSON', 'FORGERY', 'GRAND LARCENY', 'PETIT LARCENY',
       'GRAND LARCENY OF MOTOR VEHICLE', 'ASSAULT 3 & RELATED OFFENSES',
       'OFFENSES INVOLVING FRAUD', 'FRAUDS',
       'OTHER STATE LAWS (NON PENAL LA', 'BURGLARY', 'ROBBERY',
       'DANGEROUS WEAPONS', 'CRIMINAL MISCHIEF & RELATED OF',
       'THEFT-FRAUD', 'VEHICLE AND TRAFFIC LAWS', 'DANGEROUS DRUGS',
       'SEX CRIMES', 'OFF. AGNST PUB ORD SENSBLTY &', 'RAPE',
       'ADMINISTRATIVE CODE', 'INTOXICATED & IMPAIRED DRIVING',
       'OFFENSES AGAINST PUBLIC ADMINI', 'OFFENSES AGAINST THE PERSON',
       'CRIMINAL TRESPASS', 'POSSESSION OF STOLEN PROPERTY',
       'UNAUTHORIZED USE OF A VEHICLE', 'OTHER OFFENSES RELATED TO THEF',
       'ENDAN WELFARE INCOMP', 'PETIT LARCENY OF MOTOR VEHICLE',
       'NYS LAWS-UNCLASSIFIED FELONY',
       'AGRICULTURE & MRKTS LAW-UNCLASSIFIED',
       'KIDNAPPING & RELATED OFFENSES', 'FRAUDULENT ACCOSTING',
       "B

In [27]:
#df.to_csv('preprocess_311_2.csv')